In [ ]:
# default_exp tasks

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
from biocores.softwares.default import *
from biocores.softwares.bwa import Bwa
from biocores.softwares.star import Star
from create_reference.commands import *
from create_reference.utils import file_exists

In [ ]:
# export

def task_get_file(link,filename):
    if file_exists(filename):
        return 'echo {filename} exists'.format(filename=filename)
    else:
        return cmd_wget(link,filename)

def task_star_build_index(software,reference,gtf,outdir,rl):
    star = Star(software,starDefault)
    return star.cmd_build_index(outdir, reference, gtf, rl)

In [ ]:
print(task_star_build_index('STAR','genome.fasta','genome.gtf','/tmp',1))


STAR --runThreadN 8 \
         --runMode genomeGenerate  \
        --genomeDir /tmp \
        --genomeFastaFiles genome.fasta \
        --sjdbGTFfile genome.gtf \
        --sjdbOverhang 1


In [ ]:
hs_link='ftp://ftp.ensembl.org/pub/release-99/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz'
ss_link='ftp://ftp.ensembl.org/pub/release-99/fasta/sus_scrofa/dna/Sus_scrofa.Sscrofa11.1.dna.toplevel.fa.gz'
outdir='/tmp/'
species='homo_sapiens'
version='99'
filename='genome.fasta.gz'


In [ ]:
task_get_file(hs_link,filename)

'wget -c ftp://ftp.ensembl.org/pub/release-99/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz -O genome.fasta.gz'

In [ ]:
task_get_file(ss_link,filename)

'wget -c ftp://ftp.ensembl.org/pub/release-99/fasta/sus_scrofa/dna/Sus_scrofa.Sscrofa11.1.dna.toplevel.fa.gz -O genome.fasta.gz'

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 01_defaults.ipynb.
Converted 02_tasks.ipynb.
Converted 03_commands.ipynb.
Converted index.ipynb.
